### Importing all the necessary libraries

The code imports necessary libraries including scikit‑learn estimators, XGBoost, CatBoost, LightGBM, and Transformers for the ESM‑2 model.

In [17]:
import subprocess
import pandas as pd
import numpy as np
import pickle
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

### Feature Extraction via PSI Blast

Two main functions extract features for each peptide sequence:

    a). run_psiblast: Runs PSI‑BLAST on a sequence to generate a PSSM (position specific scoring matrix) and returns an averaged feature vector.
    b). extract_psiblast_features: Applies run_psiblast to each sequence in a DataFrame.
    c). extract_esm2_embeddings: Uses the ESM‑2 model to generate embeddings for each peptide sequence


In [18]:
# Function to run PSI-BLAST to generate PSSM features from a given sequence
def run_psiblast(sequence, db='nr', out_file='pssm.txt'):
    """Run PSI-BLAST to generate PSSM features"""
    # Write the query sequence to a FASTA file
    with open('query.fasta', 'w') as f:
        f.write(f">seq\n{sequence}\n")
    # Execute the PSI-BLAST command with the specified parameters
    subprocess.run(
        ["psiblast", "-query", "query.fasta", "-db", db,
         "-num_iterations", "3", "-out_ascii_pssm", out_file],
        check=True
    )
    # Load the PSSM scores from the output file; use columns 22-41 and skip header rows
    pssm = pd.read_csv(out_file, skiprows=3, delim_whitespace=True, usecols=range(22, 42))
    # Return the average scores across the sequence positions
    return pssm.mean().values  # Average PSSM scores


# Function to extract PSSM features for all sequences in a DataFrame
def extract_pssm_features(df):
    """Extract PSSM features for all sequences"""
    pssm_features = []
    # Iterate over each sequence in the DataFrame
    for seq in df['Sequence']:
        try:
            # Try extracting the PSSM features using PSI-BLAST
            pssm_features.append(run_psiblast(seq))
        except:
            # If an error occurs, append a zero-vector as a fallback
            pssm_features.append(np.zeros(20))  # Handle errors gracefully
    # Return a DataFrame of the extracted features
    return pd.DataFrame(pssm_features)


# Function to extract ESM-2 embeddings for all sequences in a DataFrame
def extract_esm2_embeddings(df):
    """Extract embeddings from ESM-2 model"""
    model_name = "facebook/esm2_t6_8M_UR50D"
    # Load the tokenizer and model from the pretrained weights
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.eval()  # Set model to evaluation mode
    
    embeddings = []
    # Iterate over each sequence in the DataFrame
    for seq in df['Sequence']:
        # Tokenize the sequence with padding and truncation
        inputs = tokenizer(seq, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            # Obtain model outputs without updating gradients
            outputs = model(**inputs)
        # Average over the sequence length to get a fixed-size embedding
        emb = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(emb)
    
    # Return a DataFrame of the embeddings
    return pd.DataFrame(embeddings)


# Load training and testing data from CSV files
train_df = pd.read_csv('train.csv', comment='#', names=['Sequence', 'Label'])
test_df = pd.read_csv('test.csv', comment='#', names=['ID', 'Sequence'])

# Extract PSSM features for training and testing data
print("Extracting PSSM features...")
X_train_pssm = extract_pssm_features(train_df)
X_test_pssm = extract_pssm_features(test_df)

# Extract ESM-2 embeddings for training and testing data
print("Extracting ESM-2 embeddings...")
X_train_esm = extract_esm2_embeddings(train_df)
X_test_esm = extract_esm2_embeddings(test_df)

# Combine both types of features for training and testing data
X_train = pd.concat([X_train_pssm, X_train_esm], axis=1)
X_test = pd.concat([X_test_pssm, X_test_esm], axis=1)

# Map training labels from {-1, 1} to {0, 1}
y_train = train_df['Label'].map({-1: 0, 1: 1})

# Remove any rows in the training set where the label is NaN
X_train = X_train[~y_train.isna()]
y_train = y_train.dropna()

# Perform a stratified train-validation split
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

# Scale features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_split)  # Fit scaler and transform training data
X_val_scaled = scaler.transform(X_val)                # Transform validation data using the same scaler
X_test_scaled = scaler.transform(X_test)              # Transform test data using the same scaler

Extracting PSSM features...
Extracting ESM-2 embeddings...


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Training all the models and choosing best

Base estimators are defined with the provided optimized parameters. Two ensembles are created:

    a). Stacking Ensemble: Combines the base estimators with LightGBM as the final estimator.
    b). Voting Ensemble: Uses soft voting with weighted probabilities.
    c). The models are stored in a dictionary for later evaluation.

In [8]:
# ----- Define Base Estimators and Ensembles -----
rf = RandomForestClassifier(n_estimators=500, max_depth=None, max_features='sqrt', 
                             class_weight='balanced', random_state=42)
xgb = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, 
                    colsample_bytree=0.8, subsample=0.8, scale_pos_weight=1.5, 
                    min_child_weight=3, random_state=42)
svm = SVC(probability=True, kernel='rbf', C=10, gamma='scale', class_weight='balanced')
catboost = CatBoostClassifier(iterations=300, depth=6, learning_rate=0.05, 
                              verbose=0, random_state=42)
lgbm = LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, 
                      class_weight='balanced', random_state=42)
gbm = GradientBoostingClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, 
                                 random_state=42)
ada = AdaBoostClassifier(n_estimators=300, learning_rate=0.05, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, random_state=42)

stacking_clf = StackingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('svm', svm),
                ('cat', catboost), ('gbm', gbm), ('ada', ada), ('mlp', mlp)],
    final_estimator=lgbm
)

voting_clf = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('svm', svm),
                ('cat', catboost), ('gbm', gbm), ('ada', ada), ('mlp', mlp)],
    voting='soft', weights=[1, 2, 1, 2, 1, 1, 1]
)

models = {
    "Random Forest": rf,
    "XGBoost": xgb,
    "SVM": svm,
    "CatBoost": catboost,
    "GBM": gbm,
    "AdaBoost": ada,
    "MLP": mlp,
    "Stacking": stacking_clf,
    "Voting": voting_clf
}

# ----- Hyperparameter Tuning for XGBoost -----
grid_params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'subsample': [0.7, 0.8, 0.9],
    'min_child_weight': [1, 3, 5]
}

print("Tuning XGBoost hyperparameters...")
random_search = RandomizedSearchCV(
    xgb, grid_params, 
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
    scoring='roc_auc', n_iter=15, random_state=42
)
random_search.fit(X_train_scaled, y_train_split)
print("Best XGBoost Params:", random_search.best_params_)
xgb_best = random_search.best_estimator_
models["XGBoost"] = xgb_best

# ----- Evaluate Models on the Validation Set -----
print("\nEvaluating models...")
best_auc = 0
best_model_name = None
best_model = None

for name, model in models.items():
    model.fit(X_train_scaled, y_train_split)
    y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    print(f"{name}: AUC = {auc:.4f}")
    if auc > best_auc:
        best_auc = auc
        best_model_name = name
        best_model = model

print(f"\nBest Model: {best_model_name} with AUC = {best_auc:.4f}")

# Save the best model as a pickle file
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
print("Best model saved as 'best_model.pkl'.")

# ----- Generate Submission using Stacking Ensemble -----
print("\nTraining stacking ensemble for submission predictions...")
stacking_clf.fit(X_train_scaled, y_train_split)
y_val_pred_proba = stacking_clf.predict_proba(X_val_scaled)[:, 1]
val_auc = roc_auc_score(y_val, y_val_pred_proba)
print(f"Stacking Ensemble Validation AUC: {val_auc:.4f}")

y_test_pred_proba = stacking_clf.predict_proba(X_test_scaled)[:, 1]
submission = pd.DataFrame({'ID': test_df['ID'], 'Label': y_test_pred_proba})
submission.to_csv('submission_test_final.csv', index=False)
print("Submission file saved as 'submission_test_final.csv'.")

Tuning XGBoost hyperparameters...
Best XGBoost Params: {'subsample': 0.7, 'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 0.7}

Evaluating models...
Random Forest: AUC = 0.7988
XGBoost: AUC = 0.8578
SVM: AUC = 0.8243
CatBoost: AUC = 0.8467
GBM: AUC = 0.8223


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost: AUC = 0.8483
MLP: AUC = 0.8242


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is depr

[LightGBM] [Info] Number of positive: 3293, number of negative: 2842
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000212 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 6135, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Voting: AUC = 0.8341

Best Model: Stacking with AUC = 0.8949
Best model saved as 'best_model.pkl'.

Training stacking ensemble for submission predictions...


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is depr

[LightGBM] [Info] Number of positive: 3293, number of negative: 2842
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000164 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 6135, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

### Hyperparameter Tuning for XGBoost
A randomized search is performed using cross-validation to tune the hyperparameters for the XGBoost model. The best estimator is updated in the models dictionary.



### Model Evaluation and Saving the Best Model
    a). Each model is evaluated on the validation set using ROC‑AUC as the performance metric. The best performing model is saved as a pickle file.
    b). Finally, the stacking ensemble is retrained on the full training set (scaled) and used to predict the probabilities on the test set. These predictions are then saved in a submission CSV file.

## Accuracy from Dumbed pickel file:

In [11]:
import sklearn
print(sklearn.__version__)

1.6.1


In [12]:
pip install scikit-learn==1.4.2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pickle
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

class Dummy:
    def __init__(self, *args, **kwargs):
        pass

# Register the missing attribute as a dummy function
import sklearn._loss._loss
sklearn._loss._loss.__pyx_unpickle_CyHalfBinomialLoss = Dummy

with open("best_model.pkl", "rb") as f:
    best_model = pickle.load(f)

# Predict on the validation set
y_val_pred = best_model.predict(X_val_scaled)

# Print various validation metrics
print("Validation Metrics for the Best Model:")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Precision: {precision_score(y_val, y_val_pred):.4f}")
print(f"Recall: {recall_score(y_val, y_val_pred):.4f}")
print(f"F1 Score: {f1_score(y_val, y_val_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred))


c:\Python312\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use a

Validation Metrics for the Best Model:
Accuracy: 0.7718
Precision: 0.7305
Recall: 0.9114
F1 Score: 0.8110

Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.61      0.71       710
         1.0       0.73      0.91      0.81       824

    accuracy                           0.77      1534
   macro avg       0.79      0.76      0.76      1534
weighted avg       0.79      0.77      0.77      1534



c:\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, 